In [ ]:
!pip install qiskit
!pip install qiskit-ibm-runtime
!pip install qiskit[visualization]
!pip install qiskit-aer
!pip install qiskit-ibm-provider

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.9/359.9 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library import Initialize
from qiskit.quantum_info import Statevector, SparsePauliOp
from qiskit.primitives import StatevectorEstimator
import numpy as np
import itertools

# Mapeo de nucleótidos
nucleotides = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

# 1) Función para codificar una cadena genética en 12 qubits
def encode_sequence_circuit(sequence: str) -> QuantumCircuit:
    num_qubits = 12
    vec = np.zeros(2**num_qubits, dtype=complex)
    amp = 1 / np.sqrt(len(sequence))

    for i, nt in enumerate(sequence):
        if i >= 2**10:
            raise ValueError("Secuencia >1024 caracteres")
        idx = (i << 2) | nucleotides[nt]
        vec[idx] = amp

    qc = QuantumCircuit(num_qubits)
    qc.initialize(vec, qc.qubits)
    return qc

# 2) Construir el proyector |φ><φ| como SparsePauliOp
def projector_from_state(state: Statevector) -> SparsePauliOp:
    mat = np.outer(state.data, state.data.conj())
    return SparsePauliOp.from_operator(mat)

# 3) Fidelity usando StatevectorEstimator
def fidelity_estimator(circuit: QuantumCircuit, target_state: Statevector) -> float:
    observable = projector_from_state(target_state)
    estimator = StatevectorEstimator()
    job = estimator.run([(circuit, observable)])
    res = job.result()

    if hasattr(res, 'values'):
        val = res.values[0]
    elif hasattr(res, '__iter__') and len(res) > 0 and hasattr(res[0], 'data') and hasattr(res[0].data, 'evs'):
        val = res[0].data.evs
    elif hasattr(res, 'data') and hasattr(res.data, 'evs'):
        val = res.data.evs
    else:
        raise RuntimeError("No pude extraer la expectativa del resultado del estimator")

    return float(np.real(val))

# 4) Cadenas de ADN (mismas longitudes)
species = {
    "human": "ACGTACGTAC",
    "chimp": "ACGTACGTAA",
    "sheep": "TGCATGCATG",
    "dog":   "TGCATGCAAA",
}

# 5) Generar y comparar pares
pairs = list(itertools.combinations(species.keys(), 2))
print("Fidelidades (Estimator):")
for a, b in pairs:
    circ_a = encode_sequence_circuit(species[a])
    circ_b = encode_sequence_circuit(species[b])
    state_b = Statevector.from_instruction(circ_b)
    f = fidelity_estimator(circ_a, state_b)
    print(f"{a} - {b}: {f:.6f}")



Fidelidades (Estimator):
human - chimp: 0.810000
human - sheep: 0.000000
human - dog: 0.010000
chimp - sheep: 0.000000
chimp - dog: 0.040000
sheep - dog: 0.640000
